In [1]:
import requests
import json

Load Metadata

In [2]:
with open("metadata.json") as f:
    metadata = json.load(f)
with open("lakes.geojson") as f:
    shape = json.load(f)

Load Satellite Data

In [3]:
response = requests.get("https://eawagrs.s3.eu-central-1.amazonaws.com/metadata/metadata.json")
satellite = response.json()

Load Datalakes Data

In [4]:
response = requests.get("https://api.datalakes-eawag.ch/selectiontables/lakes")
datalakes_lakes = response.json()
response = requests.get("https://api.datalakes-eawag.ch/datasets")
datalakes_datasets = response.json()

Create files

In [5]:
def make_bounds(shape, key):
    s = next((d for d in shape["features"] if "key" in d["properties"] and d["properties"]["key"] == key), None)
    coordinates = s["geometry"]["coordinates"][0]
    min_lon = min(coord[0] for coord in coordinates)
    max_lon = max(coord[0] for coord in coordinates)
    min_lat = min(coord[1] for coord in coordinates)
    max_lat = max(coord[1] for coord in coordinates)
    return {
      "southWest": [min_lat, min_lon],
      "northEast": [max_lat, max_lon]
    }

In [6]:
def make_bathymetry(data, datalakes_lakes):
    bathymetry = []
    if "datalakes_id" in data:
        if isinstance(data["datalakes_id"], list):
            datalakes_ids = data["datalakes_id"]
        else:
            datalakes_ids = [data["datalakes_id"]]
        for i in datalakes_ids:
            p = next((d for d in datalakes_lakes if "id" in d and d["id"] == i), None)
            if p["morphology"]:
                bathymetry.append({
                    "type": "1D",
                    "url": "https://www.datalakes-eawag.ch/lakemorphology",
                    "source": "Swisstopo Vector25",
                    "format": "JSON"
                  })
            
    if "swiss_bathy" in data:
        bathymetry.append({
        "type": "3D",
        "url": "https://www.swisstopo.admin.ch/en/height-model-swissbathy3d#swissBATHY3D---Download",
        "source": "Swisstopo swissBATHY3D",
        "format": "XYZ"
      })
    return bathymetry

In [7]:
def make_modules(data, sd):
    modules = []
    if "alplakes" in data:
        modules.append({
          "id": "threed_map",
          "title": "Temperature and Currents",
          "subtitle": "3D Model",
          "component": "map",
          "labels": {
            "topRight": "5 day forecast"
          },
          "defaults": ["3D_currents", "3D_temperature"]
        })
    if "simstrat" not in data or data["simstrat"] != False:
        modules.append({
          "id": "graph_conditions",
          "title": "Temperature, Oxygen and Ice",
          "subtitle": "1D Model",
          "component": "graph",
          "defaults": ["temperature_heatmap"]
        })
    if "sentinel3" in sd and "chla" in sd["sentinel3"]:
        modules.append({
          "id": "satellite_chla",
          "title": "Chlorophyll A",
          "subtitle": "Remote Sensing Product",
          "component": "map",
          "labels": {
            "topRight": "satelliteDatetime",
            "topLeft": "satelliteAverage"
          },
          "defaults": ["satellite_chlorophyll"]
        })
    if ("sentinel3" in sd and "Zsd_lee" in sd["sentinel3"]) or ("sentinel2" in sd and "Z490" in sd["sentinel2"]):
        modules.append({
          "id": "satellite_secchi",
          "title": "Secchi Depth",
          "subtitle": "Remote Sensing Product",
          "component": "map",
          "labels": {
            "topRight": "satelliteDatetime",
            "topLeft": "satelliteAverage"
          },
          "defaults": ["satellite_secchi"]
        })
    if ("sentinel3" in sd and "tsm_binding754" in sd["sentinel3"]) or ("sentinel2" in sd and "tsm_dogliotti665" in sd["sentinel2"]):
        modules.append({
          "id": "satellite_turbidity",
          "title": "Turbidity",
          "subtitle": "Remote Sensing Product",
          "component": "map",
          "labels": {
            "topRight": "satelliteDatetime",
            "topLeft": "satelliteAverage"
          },
          "defaults": ["satellite_turbidity"]
        })
    if "alplakes" in data:
        modules.append({
          "id": "threed_thermocline",
          "title": "Thermocline",
          "subtitle": "3D Model",
          "component": "map",
          "labels": {
            "topRight": "5 day forecast"
          },
          "defaults": ["3D_thermocline"]
        })
    if "simstrat" not in data or data["simstrat"] != False:
        modules.append({
          "id": "graph_historic",
          "title": "Historic Trends",
          "subtitle": "Models and insitu data",
          "component": "graph",
          "defaults": ["temperature_doy"]
        })
    return modules

In [14]:
def make_layers(data, sd):
    layers = []
    if "alplakes" in data:
        
        

IndentationError: expected an indented block (469947371.py, line 3)

In [13]:
home_list = []
for lake in metadata:
    data = lake.copy()
    key = lake["key"]
    satellite_data = False
    if key in satellite:
        satellite_data = satellite[key]
    data["bounds"] = make_bounds(shape, key)    
    data["bathymetry"] = make_bathymetry(data, datalakes_lakes)
    data["insitu"] = []
    data["modules"] = make_modules(data, satellite_data)
    data["layers"] = make_layers(data, satellite_data)
        
    break

NameError: name 'make_layers' is not defined

In [9]:
data

{'key': 'geneva',
 'name': {'EN': 'Lake Geneva',
  'DE': 'Genfersee',
  'FR': 'Le Léman',
  'IT': 'Lago Lemano'},
 'area': 580,
 'ave_depth': 154,
 'max_depth': 310,
 'elevation': 372,
 'latitude': 46.45,
 'longitude': 6.5,
 'flags': ['swiss', 'french'],
 'type': 'Natural',
 'volume': 89.0,
 'residence_time': 4160.0,
 'mixing_regime': 'Oligomictic',
 'geothermal_flux': 0.08,
 'trophic_state': 'Mesotrophic',
 'datalakes_id': 1,
 'swiss_bathy': True,
 'sediment_oxygen_uptake_rate': -37.5,
 'alplakes': {'3D_temperature': [{'name': 'Lausanne',
    'latlng': [46.50995, 6.61172]},
   {'name': 'Geneva', 'latlng': [46.22034, 6.1564], 'direction': 'right'},
   {'name': 'Thonon', 'latlng': [46.37547, 6.47259], 'direction': 'bottom'},
   {'name': 'Vevey', 'latlng': [46.45841, 6.84158]}],
  '3D_thermocline': [{'name': 'LéXPLORE',
    'latlng': [46.5, 6.67],
    'direction': 'left'},
   {'name': 'SHL2', 'latlng': [46.45, 6.59], 'direction': 'bottom'}]},
 'bounds': {'southWest': [46.2046, 6.1427], '

In [10]:
datalakes_lakes

[{'id': 1,
  'name': 'Lake Geneva',
  'elevation': '372.0',
  'depth': '309.0',
  'morphology': True},
 {'id': 2,
  'name': 'Lake Biel',
  'elevation': '429.0',
  'depth': '74.0',
  'morphology': True},
 {'id': 3,
  'name': 'Greifensee',
  'elevation': '435.0',
  'depth': '32.0',
  'morphology': True},
 {'id': 6,
  'name': 'Lake Aegeri',
  'elevation': '724.0',
  'depth': '83.0',
  'morphology': True},
 {'id': 7,
  'name': 'Lake Baldegg',
  'elevation': '463.0',
  'depth': '66.0',
  'morphology': True},
 {'id': 8,
  'name': 'Lake Hallwil',
  'elevation': '449.0',
  'depth': '48.0',
  'morphology': True},
 {'id': 10,
  'name': 'Upper Lake Constance',
  'elevation': '395.0',
  'depth': '251.0',
  'morphology': True},
 {'id': 12,
  'name': 'Lake Brienz',
  'elevation': '564.0',
  'depth': '259.0',
  'morphology': True},
 {'id': 14,
  'name': 'Lake Maggiore',
  'elevation': '193.0',
  'depth': '372.0',
  'morphology': True},
 {'id': 15,
  'name': 'Lake Cadagno',
  'elevation': '1921.0',
  

Satellite

In [11]:
satellite

{'ageri': {'sentinel2': ['SPM_Nechad2016_665',
   'TUR_Nechad2016_665',
   'p3qaa_zSD',
   'tsm_dogliotti665',
   'Z490'],
  'sentinel3': ['chla',
   'tsm_binding754',
   'pp_integral',
   'Zsd_lee',
   'a_ph443',
   'area_bgr',
   'bgr_whit']},
 'baldegg': {'sentinel2': ['SPM_Nechad2016_665',
   'TUR_Nechad2016_665',
   'p3qaa_zSD',
   'tsm_dogliotti665',
   'Z490'],
  'sentinel3': ['chla',
   'tsm_binding754',
   'pp_integral',
   'Zsd_lee',
   'a_ph443',
   'area_bgr',
   'bgr_whit']},
 'hallwil': {'sentinel2': ['SPM_Nechad2016_665',
   'TUR_Nechad2016_665',
   'p3qaa_zSD',
   'tsm_dogliotti665',
   'Z490'],
  'sentinel3': ['chla',
   'tsm_binding754',
   'pp_integral',
   'Zsd_lee',
   'a_ph443',
   'area_bgr',
   'bgr_whit']},
 'biel': {'sentinel2': ['SPM_Nechad2016_665',
   'TUR_Nechad2016_665',
   'p3qaa_zSD',
   'tsm_dogliotti665',
   'Z490'],
  'sentinel3': ['chla',
   'tsm_binding754',
   'pp_integral',
   'Zsd_lee',
   'a_ph443',
   'area_bgr',
   'bgr_whit']},
 'constance':

In [12]:
datalakes_lakes

[{'id': 1,
  'name': 'Lake Geneva',
  'elevation': '372.0',
  'depth': '309.0',
  'morphology': True},
 {'id': 2,
  'name': 'Lake Biel',
  'elevation': '429.0',
  'depth': '74.0',
  'morphology': True},
 {'id': 3,
  'name': 'Greifensee',
  'elevation': '435.0',
  'depth': '32.0',
  'morphology': True},
 {'id': 6,
  'name': 'Lake Aegeri',
  'elevation': '724.0',
  'depth': '83.0',
  'morphology': True},
 {'id': 7,
  'name': 'Lake Baldegg',
  'elevation': '463.0',
  'depth': '66.0',
  'morphology': True},
 {'id': 8,
  'name': 'Lake Hallwil',
  'elevation': '449.0',
  'depth': '48.0',
  'morphology': True},
 {'id': 10,
  'name': 'Upper Lake Constance',
  'elevation': '395.0',
  'depth': '251.0',
  'morphology': True},
 {'id': 12,
  'name': 'Lake Brienz',
  'elevation': '564.0',
  'depth': '259.0',
  'morphology': True},
 {'id': 14,
  'name': 'Lake Maggiore',
  'elevation': '193.0',
  'depth': '372.0',
  'morphology': True},
 {'id': 15,
  'name': 'Lake Cadagno',
  'elevation': '1921.0',
  